In [ ]:
!pip install -q gdown faiss-gpu 

In [ ]:
!gdown --id 1_9NPC8E-kt4pU6rTi5lkh91eFR_jnFav
!unzip -q /kaggle/working/wb_2D3Dretrieval_dataset.zip
!rm /kaggle/working/wb_2D3Dretrieval_dataset.zip

In [ ]:
import torch
import os
import math

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

# from kmeans_pytorch import kmeans
from PIL import Image

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def initiate_model(model_name):
    m = getattr(models, model_name)
    if model_name == 'resnet50':
        model = m(weights=models.ResNet50_Weights.DEFAULT)
    elif model_name =='vgg19':
        model = m(weights=models.VGG19_Weights.DEFAULT)
    elif model_name == 'efficientnet_b0':
        model = m(weights=models.EfficientNet_B0_Weights.DEFAULT)
        
    model.to(device)
    
    return model.eval()

model = initiate_model('resnet50') #resnet50, vgg19, efficientnet_b0

def assign_layer(model, model_name):
    if model_name == 'resnet50':
        model_embed = nn.Sequential(*list(model.children())[:-1])
        
        return model_embed

    else:
        model.classifier = model.classifier[:-1]
        
        return model
        
model_embed = assign_layer(model, 'resnet50')

def embed_image(img_path):
    img = Image.open(img_path).convert('RGB')
    img_trans = transform(img)
    img_trans = img_trans.to(device)
    img_trans = img_trans.unsqueeze(0)

    with torch.no_grad():
        embedding = model_embed(img_trans)
    return embedding

def embed_dataset(target_dir):
    dataset = {}
    for img_name in os.listdir(target_dir):
        img_path = os.path.join(target_dir, img_name)
        dataset[img_name] = embed_image(img_path)
        torch.cuda.empty_cache()
    return dataset

dataset = embed_dataset('database_2D')

def similar_images(source_file, n=None):
    source_vec = embed_image(source_file)
    
    cosine = nn.CosineSimilarity(dim=1)
    
    sim_dict = {}
    for k, v in dataset.items():
        sim = cosine(v, source_vec).item()
        sim_dict[k] = sim

    sim_dict = {k: v for k, v in sorted(sim_dict.items(), key=lambda item: item[1], reverse=True)}

    if n is not None:
        sim_dict = dict(list(sim_dict.items())[:n])

    return list(sim_dict.keys())

In [ ]:
query_mapping = {}

for x in os.listdir('queries'):
    query_mapping[x] = similar_images(os.path.join('queries', x), 5)    

In [ ]:
def compute_mrr_5(query_mapping):
    mrr_at_5 = 0
    
    for k, v in query_mapping.items():
        for i, j in enumerate(v):
            if k.split('.')[0] == j.split('.')[0]:
                mrr_at_5 += i
    
    return mrr_at_5 / len(query_mapping)

compute_mrr_5(query_mapping)